## Lib 

In [6]:
import pandas as pd
import json
import ast
from collections import Counter, defaultdict

## Load Data and convert to dataframe

In [7]:
with open('data-es-activity-coord-clean.json', 'r') as file:
    data = json.load(file)
df = pd.json_normalize(data)
print(df.columns.tolist())

['inst_numero', 'inst_nom', 'inst_adresse', 'inst_cp', 'inst_com_code', 'inst_com_nom', 'new_code', 'new_name', 'inst_actif', 'inst_etat', 'inst_date_creation', 'inst_date_etat', 'inst_date_valid', 'inst_acc_handi_bool', 'inst_enqu_date', 'inst_obs', 'inst_trans_bool', 'inst_part_bool', 'inst_part_type', 'inst_part_type_filter', 'inst_acc_handi_type', 'inst_trans_type', 'inst_siret', 'inst_uai', 'inst_hs_bool', 'equip_numero', 'equip_nom', 'equip_type_code', 'equip_type_name', 'equip_type_famille', 'equip_etat', 'equip_x', 'equip_y', 'equip_eclair', 'equip_acc_libre', 'equip_conf_bool', 'equip_ouv_public_bool', 'equip_loc_bool', 'equip_saison', 'equip_douche', 'equip_sanit', 'equip_dsp', 'equip_pmr_acc', 'equip_pmr_aire', 'equip_pmr_chem', 'equip_pmr_douche', 'equip_pmr_sanit', 'equip_pmr_trib', 'equip_pmr_vest', 'equip_pshs_aire', 'equip_pshs_chem', 'equip_pshs_sanit', 'equip_pshs_sign', 'equip_pshs_trib', 'equip_pshs_vest', 'equip_homo_date', 'equip_travaux_date', 'equip_erp_cat', 'e

In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 322777 entries, 0 to 322776
Columns: 119 entries, inst_numero to coordonnees
dtypes: float64(20), object(99)
memory usage: 293.0+ MB


In [9]:
df["equip_aps_json"][0]

'[{"activite": "1101", "nom": "Basket-Ball", "discipline": "Basketball", "caracteristiques": {"equip_aps_pratique": ["Activité praticable"], "equip_aps_niveau": "Loisir - Entretien - Remise en forme"}}, {"activite": "3701", "nom": "Handball / Mini hand / Handball de plage", "discipline": "Handball", "caracteristiques": {"equip_aps_pratique": ["Activité praticable"], "equip_aps_niveau": "Loisir - Entretien - Remise en forme"}}, {"activite": "8801", "nom": "Volley-ball / Volley-ball de plage (beach-volley) / Green-Volley", "discipline": "Volleyball", "caracteristiques": {"equip_aps_pratique": ["Activité praticable"], "equip_aps_niveau": "Loisir - Entretien - Remise en forme"}}]'

In [8]:
# Explode the 'equip_aps_code' and 'equip_aps_nom' columns
df_exploded = df.explode('equip_aps_code').explode('equip_aps_nom')

# Group by 'equip_aps_code' and 'equip_aps_nom' and count the occurrences
equip_aps_counts = df_exploded.groupby(['equip_aps_code', 'equip_aps_nom']).size().reset_index(name='occurrences')

# Sort the results by the count in descending order
equip_aps_counts_sorted = equip_aps_counts.sort_values(by='occurrences', ascending=False)

# Display the sorted results
print(equip_aps_counts_sorted)

                         equip_aps_code  \
54086                          ["7901"]   
22417                          ["2901"]   
35030                          ["6001"]   
35878                          ["6201"]   
8825                           ["1101"]   
...                                 ...   
9880   ["1401", "4401", "6604", "4501"]   
9881   ["1401", "4401", "6604", "4501"]   
65151                   ["901", "8801"]   
65152                   ["901", "8801"]   
1              ["1001", "1002", "8101"]   

                                           equip_aps_nom  occurrences  
54086                                             Tennis        39740  
22417              Football / Football en salle (Futsal)        36385  
35030                          Pétanque et jeu provencal        22389  
35878                                 Randonnée pédestre        16457  
8825                                         Basket-Ball         9001  
...                                                

In [ ]:
def count_noms(df):
    nom_counter = Counter()

    for l_nom in df['equip_aps_nom']:
        if isinstance(l_nom, str):
            try:
                nom_list = ast.literal_eval(l_nom)
            except ValueError:
                nom_list = [l_nom]  # If it's a single string, make it a list
        else:
            nom_list = l_nom if isinstance(l_nom, list) else [l_nom]  # Handle non-list iterables
        
        nom_counter.update(nom_list)

    return nom_counter

# Assuming 'df' is your DataFrame
nom_counts = count_noms(df)

# Get the most common 'nom'
most_common_nom = nom_counts.most_common(1)[0]

print(f"The most common 'nom' is '{most_common_nom[0]}' with {most_common_nom[1]} occurrences.")

# Print all unique 'nom' values and their counts
print("\nAll 'nom' values and their counts:")
for nom, count in nom_counts.items():
    print(f"'{nom}': {count}")

# Total number of unique 'nom' values
print(f"\nTotal number of unique 'nom' values: {len(nom_counts)}")

In [ ]:
# get the equip_aps_code, associated equip_aps_nom and the occurences in the dataset finally sort by most popular 

In [5]:
# if these column are empty delete the corresponding row
print(df.shape)
#df_lat_lon = df.dropna(subset=['coordonnees.lat', 'coordonnees.lon'], how='any')
#df_lat_lon.shape

(322777, 119)


In [6]:
#row_deleted = df.shape[0] - df_lat_lon.shape[0]
#row_deleted

In [7]:
def clean_dataframe(df, output_path):
    # Create a copy of the original dataframe
    df_copy = df.copy()
    
    # Get the initial number of rows
    initial_rows = len(df_copy)
    
    # Remove rows where all specified columns are None
    df_copy = df_copy.dropna(subset=['equip_aps_json', 'equip_aps_nom', 'equip_aps_code'], how='all')
    
    # Get the final number of rows
    final_rows = len(df_copy)
    
    # Print the number of rows removed
    print(f"Rows removed: {initial_rows - final_rows}")
    print(f"Rows remaining: {final_rows}")
    
     # Convert the dataframe to a list of dictionaries
    data_list = df_copy.to_dict(orient='records')
     # Save the cleaned dataframe as JSON
    with open(output_path, 'w', encoding='utf-8') as f:
        json.dump(data_list, f)
    
    print(f"Cleaned dataframe saved to {output_path}")

In [8]:
# clean_dataframe(df_lat_lon, './data-es-activity-coord-clean.json')

In [14]:
def count_by_code(c):

    lst_code = []
    count = 0
    for l_code in df['equip_aps_code']:
        code_list = ast.literal_eval(l_code)
        for code in code_list:
            if code == c:
                count += 1
            # if code not in lst_code:
               #lst_code.append(code)
    # len(lst_code), lst_code 
    print(count) 

In [15]:
count_by_code("1101")

44564


In [10]:
lst_nom = []

for l_nom in df['equip_aps_nom']:
    #print(l_nom)
    if isinstance(l_nom, str):
        try:
            nom_list = ast.literal_eval(l_nom)
        except ValueError:
            nom_list = [l_nom]  # If it's a single string, make it a list
    else:
        nom_list = l_nom  # If it's already a list, use it as is
    #print(nom_list[1])
    for nom in nom_list:
        if nom not in lst_nom:
           lst_nom.append(nom)
    # lst_nom.append(nom_list)
    #break  # Remove this if you want to process all rows
len(lst_nom), lst_nom 

(229,
 ['Basket-Ball',
  'Handball / Mini hand / Handball de plage',
  'Volley-ball / Volley-ball de plage (beach-volley) / Green-Volley',
  'Spéléologie',
  'Autres Danses',
  'Gymnastique Artistique',
  'Gymnastique volontaire',
  'Taekwondo / Hapkido / Tang-soo-do / Soo bahk do / Sin moo hapkido / Hapkido jin jung kwan',
  'Tennis de table',
  'Activités de forme et de santé',
  'Pétanque et jeu provencal',
  'Canyonisme',
  'Football / Football en salle (Futsal)',
  'Tennis',
  'Expression gymnique',
  'Escalade',
  'Deltaplane',
  'Parapente',
  'Saut',
  'Musculation',
  'Badminton, Jeu de volant',
  'Carabine (hors 300m) / Pistolet / Plateaux',
  'Dressage',
  "Saut d'obstacle",
  'Equitation',
  'Aikido / Aikibudo / Budo',
  'Karaté/Karaté jutsu/Goshin jutsu/Kobudo/Nihon Tai-jutsu/Taijitsu/Nambudo/Ninjutsu/Nunchaku/...',
  'Judo / Jujitsu / Taïso',
  'Cesta punta/Mains nues/Pala/Chistera (grande, joko garbi)/Paleta/Xare/Frontenis/Pala corta/Rebot',
  'Escrime (Epée / Fleuret / 

In [12]:
def count_noms(df):
    nom_counter = Counter()

    for l_nom in df['equip_aps_nom']:
        if isinstance(l_nom, str):
            try:
                nom_list = ast.literal_eval(l_nom)
            except ValueError:
                nom_list = [l_nom]  # If it's a single string, make it a list
        else:
            nom_list = l_nom if isinstance(l_nom, list) else [l_nom]  # Handle non-list iterables
        
        nom_counter.update(nom_list)

    return nom_counter

# Assuming 'df' is your DataFrame
nom_counts = count_noms(df)

# Get the most common 'nom'
most_common_nom = nom_counts.most_common(1)[0]

print(f"The most common 'nom' is '{most_common_nom[0]}' with {most_common_nom[1]} occurrences.")

# Print all unique 'nom' values and their counts
print("\nAll 'nom' values and their counts:")
for nom, count in nom_counts.items():
    print(f"'{nom}': {count}")

# Total number of unique 'nom' values
print(f"\nTotal number of unique 'nom' values: {len(nom_counts)}")

The most common 'nom' is 'Football / Football en salle (Futsal)' with 61356 occurrences.

All 'nom' values and their counts:
'Basket-Ball': 44564
'Handball / Mini hand / Handball de plage': 34121
'Volley-ball / Volley-ball de plage (beach-volley) / Green-Volley': 21272
'Spéléologie': 5039
'Autres Danses': 11753
'Gymnastique Artistique': 6960
'Gymnastique volontaire': 13531
'Taekwondo / Hapkido / Tang-soo-do / Soo bahk do / Sin moo hapkido / Hapkido jin jung kwan': 1059
'Tennis de table': 9339
'Activités de forme et de santé': 16757
'Pétanque et jeu provencal': 24247
'Canyonisme': 1287
'Football / Football en salle (Futsal)': 61356
'Tennis': 51550
'Expression gymnique': 5099
'Escalade': 6803
'Deltaplane': 794
'Parapente': 1485
'Saut': 5675
'Musculation': 6878
'Badminton, Jeu de volant': 12774
'Carabine (hors 300m) / Pistolet / Plateaux': 1856
'Dressage': 7607
'Saut d'obstacle': 6754
'Equitation': 11854
'Aikido / Aikibudo / Budo': 2029
'Karaté/Karaté jutsu/Goshin jutsu/Kobudo/Nihon Tai-j